In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from IPython.display import HTML
from io import StringIO
import urllib
import numpy as np

In [5]:
list(range(2008,2018+1))

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [6]:
data = []

for year in range(2008,2018+1):

    r = requests.get("https://www.jbic.go.jp/en/information/press/press-{}.html".format(year))
    soup = BeautifulSoup(r.content, 'html.parser')
    
    for table in soup.find_all('table'):
        for row in table.find_all('tr'):
            e = {}
            date = row.find_all('th')[0].get_text()
            date = date.replace('\xa0',' ')
            date = pd.to_datetime(date)
            e['date'] = date
            
            desc = row.find_all('a')[0]
            
            e['project_desc'] = desc.get_text()
            e['url'] = 'https://www.jbic.go.jp/en/information/press'+desc.get_attribute_list('href')[0]
            for item in row.find_all('li'):
                e[item.get_attribute_list('class')[0]] = item.get_text()
            data.append(e)

df = pd.DataFrame(data)

In [7]:
def get_row_value (row):
    return row[~row.isna()].values[0]

df['region'] = df.iloc[:,0:8].apply(axis=1, func=get_row_value)
df['region'] = df['region'].str.replace('Region:','')


In [8]:
m = df.iloc[:,9:18].apply(axis=1,func=lambda row: (~row.isna()).sum()).max()
m_finance = ['finance_{}'.format(i) for i in range(1,m+1)]

def get_row_finance (row):
    v = row[~row.isna()].sort_values().dropna().values
    size = int(m-v.size)
    return pd.Series(np.hstack([v,[np.nan for _ in range(int(3-v.size))]]))

df_m = df.iloc[:,9:18].apply(axis=1,func=get_row_finance)

for i in range(m):
    df[m_finance[i]] = df_m.iloc[:,i]

In [9]:

m = df.iloc[:,19:26].apply(axis=1,func=lambda row: (~row.isna()).sum()).max()
m_sector = ['sector_{}'.format(i) for i in range(1,m+1)]

def get_row_sector (row):
    v = row[~row.isna()].sort_values().dropna().values
    size = int(3-v.size)
    return pd.Series(np.hstack([v,[np.nan for _ in range(int(3-v.size))]]))

df_m = df.iloc[:,19:26].apply(axis=1,func=get_row_sector)

for i in range(m):
    df[m_sector[i]] = df_m.iloc[:,i]

In [10]:
df = df.drop(['area_africa', 'area_asia', 'area_europe', 'area_latin-caribbean',
       'area_middleeast', 'area_north-america', 'area_oceania',
       'area_other-area','finance_010', 'finance_020', 'finance_030', 'finance_040',
       'finance_050', 'finance_060', 'finance_070', 'finance_080',
       'finance_090', 'sector_environment', 'sector_infrastructure',
       'sector_machine-equipment', 'sector_manufacture-services',
       'sector_marine-aero', 'sector_others', 'sector_resources',
       'sector_smes'],axis=1)

In [11]:
df.head(1).T.reset_index()

,index,0
0,date,2009-03-27 00:00:00
1,project_desc,JBIC Signs Loan Agreement to Support Port Expa...
2,url,https://www.jbic.go.jp/en/information/press./p...
3,region,Africa
4,finance_1,Guarantees
5,finance_2,Untied Loans
6,finance_3,NaN
7,sector_1,Infrastructures
8,sector_2,Manufacturing and Services
9,sector_3,NaN


In [12]:
import datetime 

string = datetime.date.today().strftime('%Y%b%d')
df.to_excel('JBIC_project_data_{}.xlsx'.format(string))